# Replicating _Intermediary asset pricing: New evidence from many asset classes_

### Hanlu Ge and Junyuan Liu

In this Final Project, our main task is to reproduce Table 2 and Table 3 from the paper "Intermediary asset pricing: New evidence from many asset classes" and to carry out a series of extension works based on this. Our specific work is divided into the following parts:
1. Modify the primary dealer list (ticks.csv) based on real data sources.
2. Adjust the calculation methods for key ratios and macroeconomic variables in Table 2 and Table 3 according to the description in the paper.
3. Automatically generate and save the reproduced table results as .tex files, and further perform data analysis such as descriptive statistics, correlation analysis, and trend plots of factors.
4. Write additional files and implement project automation, such as the notebook, dodo.py, README file, and test files.

Through the above work, we have successfully optimized the reproduction based on the reference code, making the reproduced results extremely close to the target results while achieving clear visualization and an automated project workflow.

In [ ]:
import load_nyfed
load_nyfed.pull_nyfed_primary_dealers_list(load_nyfed.url)

import pandas as pd
import wrds
import config
from datetime import datetime
import unittest
import matplotlib.pyplot as plt
import numpy as np
import Table02Analysis
import Table02Prep

import Table03Load
import Table03Analysis
import Table03

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### Table 02

The following code reads in a manual data file that contains necessary information on primary dealers from 1960-2012 and then merges it with the CRSP Compustat Merge Linkhist table to get additional information on each, such as the SIC codes. The linkhist table is also used as the main reference table to pull the other comparison groups for the table.

In [ ]:
db = wrds.Connection(wrds_username=config.WRDS_USERNAME)

In [ ]:
import importlib
importlib.reload(Table02Prep)
importlib.reload(config)

In [ ]:
prim_dealers = Table02Prep.clean_primary_dealers_data(fname='Primary_Dealer_Link_Table3.csv')
link_hist = Table02Prep.load_link_table(fname='updated_linktable.csv')

As mentioned above, the next step is now to use the linkhist table to determine what the other comparison groups are. We use SIC codes to determine broker dealers and banks, and we make sure to exclude any firms that are already in the primary dealer group so we do not have duplicates. Below is the reference table for broker dealers, which had explicit SIC codes mentioned in the paper - banks did not have explicit SIC codes mentioned and required research.

In [ ]:
comparison_group_link_dict = Table02Prep.create_comparison_group_linktables(link_hist, prim_dealers)
comparison_group_link_dict['PD']

We then used each of the reference tables (primary dealers, broker dealers, banks, all firms in Compustat) and pulled data from the Compustat Fundamentals Quarterly table. The paper mentioned use of monthly data, but there was no apparent monthly table for financial statement data from Compustat. This led to some confusion about how these authors generated their ratios - I am wondering if they computed monthly ratios given what months came out of annual and then took the average.

Below is the dataset for broker dealers. We calculate or directly pull the values we need in our query so we don't need to do it after. It was mentioned in class this was a best practice because it would run on WRDS servers.

In [ ]:
datasets = Table02Prep.pull_data_for_all_comparison_groups(db, comparison_group_link_dict)
datasets['BD']

We then prep that data further by aggregating by year and standardizing the date to the first of the year. We also convert the datadate to a datetime column that can be sliced.

In [ ]:
prepped_datasets = Table02Prep.prep_datasets(datasets)
prepped_datasets['Banks']

We then weave in some of our analysis into the process, whenever the necessary dataset for the analysis is first available. Below is our main table of ratios, where we have computed the
$$
\frac{\text{Primary dealers amount}}{\text{Comparison group amount (less PD) + Primary dealers amount}}
$$

In [ ]:
Table02Analysis.create_summary_stat_table_for_data(datasets)
table = Table02Prep.create_ratios_for_table(prepped_datasets)
table

We create a figure that can give the reader insight into how the ratios have shifted over time for each category and comparison group. We had to clean some of the data and fill null values to have the graph look reasonable but without changing the overall shape of it too dramatically.
Lastly, we get our final table which was what we were trying to replicate from the paper. This table is then converted to LaTeX and outputted to a .tex file.

In [ ]:
formatted_table = Table02Prep.format_final_table(table)
formatted_table

In [ ]:
updated_table = Table02Prep.main(UPDATED=True)
updated_table

### Table 03

#### 3.1 Pull the quarterly dataset for primary dealers
- Start with the merged data for primary dealers from 'Table02.prim_deal_merge_manual_data_w_linktable'. The resulting dataframe 'prim_dealers' contains the list of gvkey, starting dates, and ending dates for the primary dealers during 1960-2012. 
- Use the 'fetch_data_for_tickers' function to retrieve the financial data (assets, liabilities, equity(market and book), etc.) for the tickers associated with the primary dealers from the WRDS database. The pulled items are total assets, book debt and equity, and market equity, which are needed for the calculation of market and book capital ratios.

In [ ]:
import importlib
importlib.reload(Table03Load)

In [ ]:
db.close()
db = wrds.Connection(wrds_username=config.WRDS_USERNAME)


In [ ]:
# db = wrds.Connection(wrds_username=config.WRDS_USERNAME)
#prim_dealers, _ = Table02Prep.prim_deal_merge_manual_data_w_linktable()
prim_dealers = Table02Prep.clean_primary_dealers_data(fname='Primary_Dealer_Link_Table3.csv')
dataset, _ = Table03Load.fetch_data_for_tickers(prim_dealers, db)    
dataset

#### 3.2 Prepare the dealers dataset and macro data 
- The prep_dataset function cleanses the dataset and integrates broker-dealer financial information from Flow of Funds, which is needed for calculating AEM capital ratios. The resulting prep_datast begins from a year before to 1970 for the subsequent factor and growth rate computations.
- The UPDATED parameter controls the source for broker-dealer financial data: False retrieves historical data up to 2012-12-31 from a specific URL, while True  pulls more recent data from FRED.
    - historical data link(Released at 2013 March): https://www.federalreserve.gov/releases/z1/20130307/data.htm     
    - FRED data link: https://fred.stlouisfed.org/series/BOGZ1FL664090005Q

In [ ]:
import importlib

importlib.reload(Table03) 

In [ ]:
prep_datast = Table03.prep_dataset(dataset, UPDATED=False)
prep_datast

#### 3.3 Calculate the Capital ratios and factors 
- The 'aggregate_ratios' function calculates the capital ratios - market capital ratio, book capital ratio, and AEM leverage ratio. The ratio formulas are described as below.
- The 'convert_ratios_to_factors' function  transforms the aggregated financial ratios into factors that can be used to analyze their impact on asset prices. The factors are AR(1) innovations to the market-based capital ratio of primary dealers, scaled by the lagged capital ratio.
- The resulting ratio_dataset and factors_dataset begin from a year before to 1970 for the subsequent factor and growth rate computations.

$$
\text{Market Capital Ratio}_t = \frac{\text{Market Equity}_t}{\text{Market Equity}_t + \text{Book Debt}_t}
$$
$$
\text{Book Capital Ratio}_t = \frac{\text{Book Equity}_t}{\text{Book Equity}_t + \text{Book Debt}_t}
$$
$$
\text{Leverage}_t = \frac{\text{Total Financial Assets}_t}{\text{Total Financial Assets}_t - \text{Total Liabilities}_t} \text{,  }
\text{Leverage Implied Capital Ratio}_t = \frac{1}{\text{Leverage}_t}
$$

In [ ]:
ratio_dataset = Table03.aggregate_ratios(prep_datast)
ratio_dataset

In [ ]:
import importlib

importlib.reload(Table03) 

In [ ]:
factors_dataset = Table03.convert_ratios_to_factors(ratio_dataset)
factors_dataset

#### 3.4 Process Macroeconomic Indicators
- The 'macro_variables' function fetches and processes macroeconomic indicators such as earnings-to-price ratio, unemployment rate, financial conditions index, Real GDP and GDP growth, market excess returns, market volatility. This dataset is needed for subsequent correlation analyses. The resulting macro_dataset, like the other datasets, begins from a year before to 1970 for the subsequent factor and growth rate computations.
    - E/P: Calculated as the inverse of the S&P 500's earnings-to-price ratio from Shiller's dataset, spanning from 1871 to the current month. 
    - Unemployment rate, Real GDP (level and growth): Derived from FRED and resampled on a quarterly basis. Real GDP growth is computed by comparing quarterly changes.
    - Chicago Fed National Financial Conditions Index (NFCI): Sourced from FRED and also resampled quarterly. High level means poor financial conditions. 
    - Market Excess Returns: Obtained from Fama-French research datasets, focusing on the market return over the risk-free rate, and converted to a quarterly frequency. 
    - Realized volatility: Calculated from the CRSP value-weighted index by measuring the standard deviation of quarterly returns.

In [ ]:
import importlib
importlib.reload(Table03Load)

In [ ]:
import load_fred
import importlib
importlib.reload(load_fred)

In [ ]:
import importlib

importlib.reload(Table03) 

In [ ]:
macro_dataset = Table03.macro_variables(db, from_cache=True, UPDATED=True)

In [ ]:
macro_dataset = Table03.macro_variables(db, from_cache=True)
macro_dataset

#### 3.5 Create Panel data 
- The 'create_panelA' function creates Panel A and Panel B for Table 03. Panel A focuses on the levels of financial ratios and macroeconomic variables and Panel B delves into the factors derived from the financial ratios and their growth rates.

In [ ]:
import importlib

importlib.reload(Table03) 

In [ ]:
panelA = Table03.create_panelA(ratio_dataset, macro_dataset)
panelA

In [ ]:
panelA = Table03.create_panelA(ratio_dataset, macro_dataset)
panelA

In [ ]:
panelB = Table03.create_panelB(factors_dataset, macro_dataset)
panelB

In [ ]:
panelB = Table03.create_panelB(factors_dataset, macro_dataset)
panelB

#### 3.5 Calculate correlations
- Correlation matrices for Panel A and Panel B are calculated to examine the relationships between financial ratios, macroeconomic variables, and factors. 

In [ ]:
import importlib

importlib.reload(Table03) 


In [ ]:
correlation_panelA = Table03.calculate_correlation_panelA(panelA)
correlation_panelA

In [ ]:
correlation_panelA = Table03.calculate_correlation_panelA(panelA,UPDATED=True)
correlation_panelA

In [ ]:
correlation_panelB = Table03.calculate_correlation_panelB(panelB)
correlation_panelB

In [ ]:
correlation_panelB = Table03.calculate_correlation_panelB(panelB,UPDATED=True)
correlation_panelB

In [ ]:
import importlib

importlib.reload(Table03Analysis) 

In [ ]:
Table03Analysis.plot_figure02(ratio_dataset, correlation_panelA, UPDATED=False)

#### 3.6 Generate summary outputs and export to LaTeX
- We summarize our findings into tables for Panel A and B, alongside a figure that illustrates how the capital ratios have shifted over time. All time-series are standardized to zero mean and unit variance for illustration. 
- Lastly, we get our final table which was what we were trying to replicate from the original paper. This table is converted to LaTeX and outputted to a .tex file.
- Below is the figure that we created with UPDATED=True which incorporates the most recent data (also incorporating data starting from 2013). This can be compared with the 'AEM_level.png' from the authors' presentation slides.

In [ ]:
from IPython.display import Image
Image(filename='../_output/updated_table03_figure.png', width=800)